In [ ]:
import cv2 
import numpy as np
import glob
import os
import shutil
import matplotlib.pyplot as plt

import bat_functions as kbf

In [ ]:
def simple_process_frame(image_files, first_frame, bat_thresh, bat_area,
                        background_array_size, focal_frame_ind):

    images = []
    for file in image_files[first_frame:background_array_size+first_frame]:
        image = cv2.imread(file)
        images.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    images = np.stack(images)
    # Here taking blue channel. Should be changed is images already 2D
    background_sum = np.sum(images[:,:,:,2],0, dtype=np.int16)


    bat_centers, bat_areas, bat_contours, rect_angles, bat_sizes, bat_rects, bat_thresh, binary = kbf.process_frame(
        images, focal_frame_ind, bat_thresh, background_sum, bat_area, debug=True)
    
    return {'bat_centers': bat_centers,
            'bat_areas': bat_areas,
            'bat_contours': bat_contours,
            'rect_angles': rect_angles,
            'bat_sizes': bat_sizes,
            'bat_rects': bat_rects,
            'bat_thresh': bat_thresh,
            'binary': binary}

In [ ]:
file = '.../general-detection/functions/koger_tracking.py'
shutil.copy(file, ".../demo-detection")

In [ ]:
image_folder = ".../demo-detection/frames" 

In [ ]:
first_frame = 50
background_array_size = 31
background_array_ind = 15
bat_thresh = .1
bat_area = 1    

image_files = sorted(glob.glob(image_folder + '/*.jpg'))

output = simple_process_frame(image_files, first_frame, 
                              bat_thresh, bat_area,
                              background_array_size, 
                              background_array_ind)

frame_ind = first_frame + background_array_ind
frame = cv2.imread(image_files[frame_ind])
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
circled_image = kbf.draw_circles_on_image(frame, 
                                      output['bat_centers'], 
                                      output['bat_sizes'], 
                                      rects=output['bat_rects'])

circled_binary_image = kbf.draw_circles_on_image(output['binary'].astype(np.uint8), 
                                             output['bat_centers'], 
                                             output['bat_sizes'], 
                                             rects=output['bat_rects'])

plt.figure(figsize=(20,10))
plt.imshow(circled_image)

plt.figure(figsize=(20,10))
plt.imshow(circled_binary_image)

In [ ]:
def process_video(process_info):
    
    image_folder = process_info['image_folder']
    first_frame = process_info['first_frame']
    last_frame = process_info['last_frame']
    save_every_n_frames = process_info['save_every_n_frames']
    save_folder = process_info['base_save_folder']
    bat_thresh = process_info['bat_thresh'] 
    bat_area_thresh = process_info['bat_area_thresh']



    print('processing', image_folder)

    try:
        os.makedirs(save_folder,  exist_ok=True)
    except OSError:
        pass

    image_files = sorted(glob.glob(image_folder + '/*.jpg'))

    # Use array size (num_frames, height, width, channels)
    # All we need other arrays for is averaging for background subtration
    # If averaging over 15 frames forward and back, num_frames = 31
    # image_ind keeps tracks of focal frame
    # image_ind - 15 is oldest frame
    # after each step replace the oldest frame, and increment image_ind

    # Different for first 15 frames:
    # Just increment image_ind but don't replace any frames

    array_size = 31

    images = []

    for file in image_files[first_frame:array_size+first_frame]:
        image = cv2.imread(file)
        images.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    images = np.stack(images)
    
    background_sum = np.sum(images[:,:,:,2], 0, dtype=np.int16)

    image_ind = 0
    total_frames = len(image_files)


    if last_frame is None:
        last_frame = total_frames
    
    if save_every_n_frames is not None:
        save_frames = True
        output_frames_folder = os.path.join(base_save_folder,
                                           'output_frames')
        os.makedirs(output_frames_folder, exist_ok=True)
    else:
        save_frames = False

    positions_list = []
    areas_list = []
    rects_list = []
    angles_list = []

    for image_num in range(first_frame, last_frame):
        bat_centers, bat_areas, bat_contours, rect_angles, bat_sizes, bat_rects, bat_thresh = kbf.process_frame(
            images, image_ind, bat_thresh, background_sum, bat_area_thresh)
        
        if save_frames and (image_num % save_every_n_frames == 0):
            if bat_centers is not None:
                circled_image = kbf.draw_circles_on_image(
                    images[image_ind].copy(), bat_centers, 
                    bat_sizes, rects=bat_rects)
                circled_image = cv2.cvtColor(circled_image ,cv2.COLOR_BGR2RGB)
            else:
                circled_image = images[image_ind].copy()
            head, tail = os.path.split(image_files[image_num])
            tail = tail.split('_')
            new_tail = ''
            for tail_part in tail[:-1]:
                new_tail = new_tail + tail_part + '_'
            new_tail = new_tail + ('new03-{0:04d}-rects.jpg').format(image_num)
            new_image_file = os.path.join(output_frames_folder, new_tail)
            cv2.imwrite(new_image_file, circled_image)
            
        if bat_centers is None:
            # Too dark
            break

        
        positions_list.append(bat_centers)
        areas_list.append(bat_areas)
        rects_list.append(bat_rects)
        angles_list.append(rect_angles)

        if image_num % 500 == 0:
            positions_file = os.path.join(save_folder, 'positions.npy')
            areas_file = os.path.join(save_folder, 'areas.npy')
            angles_file = os.path.join(save_folder, 'angles.npy')
            rects_file = os.path.join(save_folder, 'rects.npy')
            np.save(positions_file, positions_list)
            np.save(areas_file, areas_list)
            np.save(rects_file, rects_list)
            np.save(angles_file, angles_list)

        images, image_ind, background_sum = kbf.update_circular_image_array(
            images, image_ind, image_files, image_num, background_sum)

    positions_file = os.path.join(save_folder, 'positions-final.npy')
    areas_file = os.path.join(save_folder, 'areas-final.npy')
    angles_file = os.path.join(save_folder, 'angles-final.npy')
    rects_file = os.path.join(save_folder, 'rects-final.npy')
    np.save(positions_file, positions_list)
    np.save(areas_file, areas_list)
    np.save(rects_file, rects_list)
    np.save(angles_file, angles_list)

In [ ]:
base_save_folder = '.../demo-detection/' 
image_folders = ['.../demo-detection/frames']

process_dicts = []
for image_folder in image_folders:
    process_dicts.append(
        {'image_folder': image_folder,
         'first_frame': 0,
         'last_frame': None,  # Set to None to process whole video
         'save_every_n_frames': 20,
         'base_save_folder': base_save_folder,
         'bat_thresh': .1,
         'bat_area_thresh': 10
        })
print('{} dicts created.'.format(len(process_dicts)))

In [ ]:
from multiprocessing import Pool
import time

process_video(process_dicts[0])

# if __name__ == '__main__':
#     t0 = time.time()
#     with Pool(processes=1) as pool:
#         pool.map(process_video, process_dicts[:])
#     print('total time:', time.time() - t0)